In [1]:
import pandas as pd

In [2]:
import pywikibot as pb
site = pb.Site("wikidata", "wikidata")
repo = site.data_repository()

In [3]:
from bs4 import BeautifulSoup
import requests
import dateparser

In [ ]:
!pip install xlrd

In [ ]:
!pip install XlsxWriter

In [ ]:
def create_item(site, label_dict):
    new_item = pb.ItemPage(site)
    new_item.editLabels(labels=label_dict, summary="Creating Red List item")
    return new_item

In [ ]:
def set_claim(item, property, string_data):
    clm = new_item.get()['claims'][property][0]
    clm_trgt = clm.getTarget()
    print(clm_trgt)
    if clm_trgt != string_data :
        claim = pb.Claim(repo, property)
        claim.setTarget(string_data)
        item.addClaim(claim, bot=False, summary="Test")
        return claim
    else :
        return None

In [47]:
red_list_df = pd.read_excel("red2.xlsx")

In [48]:
red_list_df.columns

Index(['Número', 'Nombre', 'Artículo en español', 'Wikidata ID', 'Latitud',
       'Longitud', 'Localidad', 'Localidad ID', 'Municipio', 'Municipio ID',
       'Provincia', 'Comunidad autónoma', 'Tipo', 'Tipo 2', 'Fecha inclusión',
       'Fecha salida', 'Estatus retirada', 'Descripción', 'URL', 'Estatus',
       'Categoría', 'Imagen'],
      dtype='object')

In [ ]:
es_site = pb.Site('es', 'wikipedia')
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    municipality = row["Municipio"]
    print ("-------------")
    if not pd.notnull(municipality) :
        #print ("++++++++++ Void")
        output_df = output_df.append(row, ignore_index=True) 
        continue
    q_ids = []
    for municipality in municipality.split('|'):
        municipalities = [municipality,
                         "{0} ({1})".format(municipality, row["Provincia"]),
                         "{} (España)".format(municipality),
                         "{} (municipio)".format(municipality),                     ]
        found = False
        q_id = ''
        for m in municipalities :
            try :
                page = pb.Page(es_site, m)
                item = pb.ItemPage.fromPage(page)
                if item.claims and 'P31' in item.claims:
                    instances_of = [claim.getTarget().getID() for claim in item.claims['P31']]
                    if "Q2074737" in instances_of or "Q33146843" in instances_of or "Q5055981" in instances_of:
                        q_id = item.getID()
                        found = True
                        break
                else :
                    #print ("Not Found")
                    pass
            except Exception as e:
                #print ("{1} ({0})".format(row["Nombre"], e))
                pass
        if found :
            q_ids.append(q_id)
            #print ("'{0} ({1})'".format(municipality, q_id))
        #else :
            #print ("++++++++++ {}".format(municipality))
    row["Municipio ID"] = '|'.join(q_ids)
    print ('|'.join(q_ids))
    output_df = output_df.append(row, ignore_index=True) 

-------------
Q559221
-------------
Q490472
-------------
Q836948
-------------
Q5818
-------------
Q15678
-------------
Q490472
-------------
Q8818
-------------
Q8818
-------------
Q1648851
-------------
-------------
Q1443946
-------------
Q1442470
-------------
Q1628353
-------------
-------------
Q177501
-------------
Q951950
-------------
Q15679
-------------
Q1630794
-------------
Q830147
-------------
Q12233
-------------
Q490472
-------------
Q485329
-------------
Q8818
-------------
Q1438879
-------------
Q1773030
-------------
Q162615
-------------
Q162615
-------------
Q162615
-------------
Q162615
-------------
Q162615
-------------
Q162615
-------------
Q162615
-------------
Q8818
-------------
-------------
Q1049602
-------------
-------------
Q8717
-------------
Q15695
-------------
Q46940
-------------
Q329984
-------------
Q478178
-------------
Q1651722
-------------
Q1635901
-------------
Q8851
-------------
Q1442637
-------------
Q609827
-------------
Q1372697
-----

Q1645908
-------------
Q834281
-------------
Q1780360
-------------
Q1652984
-------------
Q1650635
-------------
Q1769672
-------------
Q1764548
-------------
Q2314270
-------------
Q1641146
-------------
Q1641508
-------------
Q1406727
-------------
Q47475
-------------
Q1657014
-------------
Q487786
-------------
Q1773034
-------------
Q1773034
-------------
Q2572251
-------------
Q49581
-------------
Q604909
-------------
Q818363
-------------
Q1778599
-------------
Q1607299
-------------
Q372556
-------------
Q1452684
-------------
Q748790
-------------
Q2807
-------------
Q490472
-------------
Q1769076
-------------
Q1916446
-------------
Q49581
-------------
Q2041849
-------------
Q1907342
-------------
Q984666
-------------
Q1368923
-------------
Q821860
-------------
Q329984
-------------
Q8810
-------------
Q592681
-------------
Q668384
-------------
Q1676138
-------------
Q1630718
-------------
Q988834
-------------
Q15695
-------------
Q1645908
-------------
Q1640382
------

In [12]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if pd.notnull(row["Wikidata ID"]) :
        print ("--------------")
        item = pb.ItemPage(repo, row["Wikidata ID"])
        item.get()
        commons_cat = None
        if item.sitelinks and "commonswiki" in item.sitelinks and "Category:" in item.sitelinks["commonswiki"]:
            print (item.sitelinks["commonswiki"])
            commons_cat = item.sitelinks["commonswiki"].split(':')[1].strip()
            
        if item.claims and 'P373' in item.claims: # instance of
            print(item.claims['P373'][0].getTarget())
            commons_cat = item.claims['P373'][0].getTarget()

        if commons_cat != None :
            row["Categoria"] = "Category:{0}".format(commons_cat)

    output_df = output_df.append(row, ignore_index=True)        

--------------
Acueducto de Albatana
--------------
--------------
Category:Alcázar de Toro
Alcázar de Toro
--------------
--------------
--------------
--------------
Alqueria del Moro
--------------
Alqueria del Pi d'Alfafar
--------------
Category:Palacio de Chiloeches, Santoña
Palacio de Chiloeches, Santoña
--------------
Category:Church of San Miguel, Sasamón
Church of San Miguel, Sasamón
--------------
--------------
--------------
Category:Tower of Valmojado
Tower of Valmojado
--------------
Category:Bahía de Santander
Bahía de Santander
--------------
--------------
Category:Cabanyal-Canyamelar
Cabanyal-Canyamelar
--------------
Category:Batería de Cenizas
Batería de Cenizas
--------------
--------------
--------------
--------------
Category:Capilla de San José, Sevilla
Capilla de San José (Sevilla)
--------------
Cartuja de Nuestra Señora de las Fuentes
--------------
--------------
Cartuja de Vall de Cristo
--------------
--------------
--------------
--------------
Category

Ermita de Guía in Jerez de la Frontera
--------------
Category:Hermitage of Virgen de la Oliva, Patones
Hermitage of Virgen de la Oliva, Patones
--------------
Hermitage of Gañarul
--------------
Category:Hermitage of Nuestra Señora de la Calzada, Brías
Hermitage of Nuestra Señora de la Calzada, Brías
--------------
Category:Ermita de San Ambrosio (Barbate)
Ermita de San Ambrosio (Barbate)
--------------
Category:Hermitage of San Juan de Socueva
Hermitage of San Juan de Socueva
--------------
--------------
Category:San Pedro de la Mata, Casalgordo
San Pedro de la Mata, Casalgordo
--------------
Category:Hermitage of San Pelayo, Perazancas de Ojeda
Hermitage of San Pelayo, Perazancas de Ojeda
--------------
--------------
Category:Ermita de Santa Ana, El Toboso
Ermita de Santa Ana, El Toboso
--------------
--------------
Ermita del Calvari de Canet lo Roig
--------------
Category:Ermita del Santo Cristo (Talaván)
Ermita del Santo Cristo (Talaván)
--------------
Hermitage of San Miguel 

--------------
--------------
Castell de Xelva
--------------
--------------
Palacio de Nava, San Cristóbal de La Laguna
--------------
--------------
Category:Palace of Tormaleo
Palace of Tormaleo
--------------
--------------
Category:Palace of Valsaín
Palace of Valsaín
--------------
--------------
--------------
Category:Conde Luna Palace
Conde Luna Palace
--------------
Category:Palacio del Infantado o del Condestable, Colindres
Palacio del Infantado o del Condestable, Colindres
--------------
Marqués de Revilla's Palace, Navares de las Cuevas
--------------
Guenduláin
--------------
Category:Mosteiro de San Paio de Diomondi
Mosteiro de San Paio de Diomondi
--------------
Category:Palacio de los Zúñiga (Curiel de Duero)
Palacio de los Zúñiga (Curiel de Duero)
--------------
--------------
Doñana National Park
--------------
Category:Peñalcazar
Peñalcazar
--------------
Mosteiro de San Xulián de Moraime
--------------
--------------
Old town hall of Laredo
--------------
Category:A

In [20]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if pd.notnull(row["Wikidata ID"]) :
        print ("--------------")
        item = pb.ItemPage(repo, row["Wikidata ID"])
        item.get()
        image = None

        if item.claims and 'P18' in item.claims: # instance of
            image = item.claims['P18'][0].getTarget().title()
            print(image)

        if image != None :
            row["Imagen"] = image

    output_df = output_df.append(row, ignore_index=True)        

--------------
File:Acueducto de Albatana.jpg
--------------
--------------
File:Toro Castillo 900.jpg
--------------
--------------
--------------
--------------
File:Alqueria del Moro, Benicalap.JPG
--------------
File:Alquería del Pi 3.JPG
--------------
File:Cantabria Santoña Chiloeches lou.JPG
--------------
File:Sasamon Burgos Arco San Miguel.jpg
--------------
File:Tajo-x.jpg
--------------
--------------
File:Atalaya valmojado.jpg
--------------
File:Landsat Bahia Santander.jpg
--------------
--------------
File:Carrer del Rosari, Cabanyal.JPG
--------------
File:Indgangen til batteriet.jpg
--------------
--------------
--------------
File:Canal romano de Llamas.jpg
--------------
File:Sevilla - Capilla de San Jose 01.jpg
--------------
File:Sariñena-Cartuja de Monegros 13.JPG
--------------
File:Aniago 27.jpg
--------------
File:Valdecristo2.jpg
--------------
--------------
--------------
--------------
--------------
File:Pal guevara.jpg
--------------
--------------
File:Ca

--------------
File:Fachada principal estación San Sebastián.jpg
--------------
File:Fundicion Averly Zaragoza 2.jpg
--------------
--------------
File:Fàbrica de La Ceramo, Benicalap.JPG
--------------
--------------
--------------
File:Vista aérea del Frontón Beti-Jai (Madrid).jpg
--------------
File:Puente Concepcion.jpg
--------------
File:Fuerte de San Cristóbal en Badajoz.jpg
--------------
File:Acantilados de la Alcazaba, chalets, Fuerte de Victoria Grande y Fuerte del Rosario, Melilla.png
--------------
File:Galeakofortea.jpg
--------------
File:Aranjuez Hospital San Carlos.jpg
--------------
File:Gotor - Convento de Nuestra Señora de la Consolación - Fachada 01.JPG
--------------
File:VILLAESCUSA DE PALOSITOS BIC JUNIO 2012 (1).JPG
--------------
--------------
File:Iglesia de Calcena, Zaragoza.jpg
--------------
File:Iglesia Villar de Matacabras (Madrigal,Ávila).JPG
--------------
--------------
File:Huidobroigl.jpg
--------------
File:SCosmeDamianEncio.jpg
--------------
Fil

--------------
File:Torre de Ruerrero 001.jpg
--------------
File:Cast Obano-Luna 001 (cropped).jpg
--------------
File:Festungsturm in San Andrés.JPG
--------------
--------------
--------------
File:Torre del Xiprer 1.jpg
--------------
File:Torre Lodeña-13.JPG
--------------
File:Torres de Campo (Bores, Liébana).jpg
--------------
--------------
--------------
File:Abadía de la Santa Cruz del Valle de los Caídos (Comunidad de Madrid) España (3194706145).jpg
--------------
File:Venta de Borondo - Detalle Torre (Junio 2016).jpg
--------------
File:Villa Calamari 01.JPG
--------------
--------------
--------------
File:San esteban2.JPG
--------------
File:Otíñar - Vegas Altas.JPG
--------------
File:Bernorio.JPG
--------------
File:Olleros de Pisuerga - Monte Cildá 001.JPG
--------------
File:Tiermes.jpg
--------------


In [ ]:
top_categories = ['Patrimonio industrial', 
                  'Patrimonio Religioso', 
                  'Patrimonio Militar', 
                  'Patrimonio Civil', 
                  'Patrimonio Natural', 
                  'Patrimonio Arqueológico']
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    print ("-----------------------------")
    url = row['URL']
    text = requests.get(url).text
    table = BeautifulSoup(text, 'html.parser').find("div", {"class": "post-meta"})
    typology = [i.text for i in table.find_all('li') if "Tipología" in i.text][0]
    categories = [i.strip() for i in typology.replace("Tipología: ", "").split(',')]
    type1_categories = [cat for cat in categories if cat in top_categories and cat != 'Retirados de la lista']
    type2_categories = [cat for cat in categories if cat not in top_categories and cat != 'Retirados de la lista']
    row['Tipo 2'] = '|'.join(type2_categories)
    row['Tipo'] = '|'.join(type1_categories)
    
    start = [i.text for i in table.find_all('li') if "inclusión" in i.text][0]
    start_date = dateparser.parse(start.replace("Fecha de inclusión: ", "").replace(", ", " de "))
    print (start_date.strftime("%Y-%m-%d"))
    row["Fecha inclusión"] = start_date.strftime("%Y-%m-%d")

    try:
        end = [i.text for i in table.find_all('li') if "retirada" in i.text][0]
        end_date = dateparser.parse(end.replace("Fecha de retirada: ", "").replace(", ", " de "))
        print (end_date.strftime("%Y-%m-%d"))
        row["Fecha salida"] = end_date.strftime("%Y-%m-%d")
    except :
        pass

    output_df = output_df.append(row, ignore_index=True)

In [46]:
writer = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Sheet1')
writer.save()